<a href=https://docs.google.com/uc?id=1xbyIyDFmryHaZ4HhVNDBeoMCZGyXCf10 target="_blank"><img src=https://docs.google.com/uc?id=1xbyIyDFmryHaZ4HhVNDBeoMCZGyXCf10 
width="150" border="10" /></a>

# Blue Brain Nexus - A knowledge graph for data-driven science - Part 2

## 5 - Exercises Part 2

The following exercises guide you through the individual steps to help you bring the experimental data from Part 1 into Nexus and search for specific data types.

<a href=https://docs.google.com/uc?id=14JkSRhwXlYQPAzqr9lEzwgcCQXOfAoOA target="_blank"><img src=https://docs.google.com/uc?id=14JkSRhwXlYQPAzqr9lEzwgcCQXOfAoOA 
width="300" border="10" /></a>

### 5-0 Installation of Pyxus, import of relevant classes and client setup

Nexus exposes a RESTful interface over HTTP(S). To faciliate API interactions, the Nexus client [Pyxus](https://github.com/HumanBrainProject/pyxus) can be used to access and manage resources in Nexus.

**Execute the following code line to install Pyxus and other dependencies:**

In [ ]:
!pip install git+https://github.com/HumanBrainProject/pyxus@v0.5.1 pyyaml requests pygments

**Import relevant Pyxus classes and required Python modules:**

In [ ]:
import requests
import yaml
import json
from pygments import highlight
from pygments.lexers import JsonLdLexer
from pygments.formatters import TerminalFormatter

from pyxus.client import NexusClient
from pyxus.resources.entity import (Organization, Domain, Schema, Instance)
from openid_http_client.auth_client.access_token_client import AccessTokenClient

**Helper function to format JSON data in a more reader-friendly way:**

In [ ]:
def pprint(string):
    json_obj = json.loads(string)
    json_str = json.dumps(json_obj, indent=2)
    lexer = JsonLdLexer()
    print(highlight(json_str, lexer, TerminalFormatter()))

#### Grab a token here (use your Collab credentials to log in): 

https://bbp-nexus.epfl.ch/staging/v0/oauth2/authorize?realm=HBP

Copy the token (the string of numbers and letters between the quotation marks) and paste it as the value of the **token** variable below:


<font color='red'>NOTE</font>: Replace everything within double quotes below with your token (including the "<" and ">" sign).

In [ ]:
token = "<INSERT YOUR TOKEN HERE>"
auth_client = AccessTokenClient(token)

#### Setup your Nexus client by executing the code below

In [ ]:
client = NexusClient(scheme="https", 
                     host="bbp-nexus.epfl.ch", 
                     prefix="staging/v0", 
                     alternative_namespace="https://bbp-nexus.epfl.ch",
                     auth_client=auth_client)

### 5-1 Create a custom domain on Nexus using Pyxus

In this tutorial, you will be working in the "sandbox" organization of Nexus. Nested inside organizations on Nexus are domains. To set up your own domain, input a custom name and decription for your domain below:

In [ ]:
organization_name = "sandbox"
domain_name = "<PUT YOUR DOMAIN NAME HERE>" # use lower case characters
domain_description = "<PUT YOUR DOMAIN DESCRIPTION HERE>"

With the provided domain name and description, you can **create your domain** on Nexus:

In [ ]:
your_domain = Domain.create_new(organization_name, domain_name, domain_description) 
client.domains.create(your_domain)

You can access your domain identifier through the link provided after executing the code below:

In [ ]:
your_domain = client.domains.read(organization_name, domain_name)
print(your_domain.data["@id"])

You can also check out your newly created domain in the [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### 5-2 Create and publish schemas for Subject, Neuron and Dataset

Now that you have created your own domain in Nexus, you can create the **schemas for Subject, Neuron and Dataset**. Copy-paste the three schemas from week4-notebook-part-1.ipynb (from the [JSON editor online](https://jsoneditoronline.org/)) as the value of the respective variable below (replace everything, including the quotation marks):

In [ ]:
subject_schema = "<PASTE SUBJECT SCHEMA HERE>"

In [ ]:
neuron_schema = "<PASTE NEURON SCHEMA HERE>"

In [ ]:
dataset_schema = "<PASTE DATASET SCHEMA HERE>"

**Repeat the following steps** for the Subject, Neuron and Dataset schema (adjust the schema_name and content variables accordingly):

-----

In [ ]:
schema_name = "subject"
schema_version = "v0.1.0"
content = subject_schema

Run the following code to create a schema and store it in Nexus:

In [ ]:
schema = Schema.create_new(organization=organization_name, 
                           domain=domain_name, 
                           schema=schema_name,
                           version=schema_version, 
                           content=content)
client.schemas.create(schema)

Access the schema identifier of the schema you have just created:

In [ ]:
schema = client.schemas.read(organization=organization_name, 
                             domain=domain_name, 
                             schema=schema_name, 
                             version=schema_version)
print("The schema identifier is {}".format(schema.data["@id"]))

To be able to submit data against your schema, it has to be published. To publish your schema, run the following code:

In [ ]:
client.schemas.publish(entity=schema,publish=True)

Check out your newly created and published schemas in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### 5-3 Create instances with prepared payloads

Now that the three schemas needed for our data are stored and published, we can validate our data against those schemas and store them in Nexus. Copy-paste the data payloads from Part 1 and paste them as the value of the respective variable below (replace everything, including the quotation marks):

<font color='red'>NOTE</font>: The value for the **"@id"** key of the **"wasDerivedFrom"** property in the neuron_metadata, morphology_metadata and electrophysiology_metadata need to be replaced with the Nexus identifier once the referenced data instance has been created (e.g. use the Nexus identifier of the subject as the value of the "@id" key of the "wasDerivedFrom" property in the neuron metadata; the Nexus identifier of your instances can be accessed through Nexus Explorer):

In [ ]:
subject_metadata = "<PASTE SUBJECT DATA HERE>"

In [ ]:
neuron_metadata = "<PASTE SUBJECT DATA HERE>"

In [ ]:
morphology_metadata = "<PASTE MORPHOLOGY DATASET DATA HERE>"

In [ ]:
electrophysiology_metadata = "<PASTE ELECTROPHYSIOLOGY DATASET DATA HERE>"

**Repeat the following steps** for the Subject, Neuron, Morphology and Electrophysiology metadata (adjust the schema_name and content variables accordingly, e.g. for the neuron metadata, change the schema_name variable value to "neuron"):

-----

Run the following code to create your instances and store them in Nexus:

In [ ]:
subject_instance = Instance.create_new(organization=organization_name, 
                               domain=domain_name, 
                               schema="subject", 
                               version="v0.1.0", 
                               content=subject_metadata)
client.instances.create(subject_instance)

In [ ]:
neuron_instance = Instance.create_new(organization=organization_name, 
                               domain=domain_name, 
                               schema="neuron", 
                               version="v0.1.0", 
                               content=neuron_metadata)
client.instances.create(neuron_instance)

In [ ]:
morphology_instance = Instance.create_new(organization=organization_name, 
                               domain=domain_name, 
                               schema="dataset", 
                               version="v0.1.0", 
                               content=morphology_metadata)
client.instances.create(morphology_instance)

In [ ]:
electrophysiology_instance = Instance.create_new(organization=organization_name, 
                               domain=domain_name, 
                               schema="dataset", 
                               version="v0.1.0", 
                               content=electrophysiology_metadata)
client.instances.create(electrophysiology_instance)

Check out your newly created data instances in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### 5-4 Update the latest instance

After data has been stored in Nexus, it can be updated (e.g. if you want to provide a more detailed description or correct a typo). Take the payload of the last dataset instance you have created and update its description in the [JSON editor online](https://jsoneditoronline.org/). Paste the updated metadata as the value of the variable "updated_electrophysiology_instance_data" below:

In [ ]:
updated_electrophysiology_instance_data = "<PASTE THE UPDATED INSTANCE DATA HERE>"

Using the updated payload, you can update your instance on Nexus:

In [ ]:
electrophysiology_instance.data = updated_electrophysiology_instance_data
client.instances.update(entity=electrophysiology_instance)

Check out your updated data instance in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### 5-5 Attach binaries to the morphology and electrophysiology dataset instance

Nexus allows one to store metadata as well as file attachments. The following steps help you attach the provided morphology file (.ASC file format) and the electrophysiology recording file (.IBW file format). Please provide the correct filenames (as found in the labook) and Nexus identifiers (accessible through Nexus Explorer > Identifier on the respective dataset instances) as the values of the variables below: 

In [ ]:
filename_morphology = "n180301-ephys.ibw"
filename_electrophysiology = "n180301-morpho.ASC"

morphology_dataset_id = morphology_instance.data["@id"]
electrophysiology_dataset_id = electrophysiology_instance.data["@id"]

morphology_dataset_revision = morphology_instance.data["nxv:rev"]
electrophysiology_dataset_revision = electrophysiology_instance.data["nxv:rev"]

Run the following code to attach the morphology file to the morphology dataset instance on Nexus:

In [ ]:
base = 'https://github.com/NataliBarros/InSilicoNeuroscienceCourse/raw/master/'
file_morphology = "{}{}".format(base,filename_morphology)
r = requests.get(file_morphology)
morphology_file =  r.content

url = "{}/attachment?rev={}".format(morphology_dataset_id, morphology_dataset_revision)
morphology = {'file': morphology_file}
response = requests.put(url, files=morphology, 
                        headers={"Authorization": "Bearer {}".format(token)})

if response.status_code < 400:
    response_text = yaml.load(response.text)
    pprint(json.dumps(response_text))
else:
     print("Status code: {}".format(response.content))

Run the following code to attach the electrophysiology file to the electrophysiology dataset instance on Nexus:

In [ ]:
base = 'https://github.com/NataliBarros/InSilicoNeuroscienceCourse/raw/master/'
file_electrophysiology = "{}{}".format(base,filename_electrophysiology)
r = requests.get(file_electrophysiology)
electrophysiology_file =  r.content

url = "{}/attachment?rev={}".format(electrophysiology_dataset_id, electrophysiology_dataset_revision)
electrophysiology = {'file': electrophysiology_file}
response = requests.put(url, files=electrophysiology, 
                        headers={"Authorization": "Bearer {}".format(token)})

if response.status_code < 400:
    response_text = yaml.load(response.text)
    pprint(json.dumps(response_text))
else:
     print("Status code: {}".format(response.content))

Check out your datasets with attachments using [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### 5-6 Query Nexus to get instances of type Neuron (spanning all the domains in the sandbox organization)

Now that you have created your schemas and validated real data instances against them, you can query Nexus to retrieve specific data. One example is to query for specific types. Since not only you but also your fellow students have put data into Nexus, you could e.g. try and filter data in Nexus by the type nsg:Neuron to retrieve all the data instances of that type. The query filter would look as follows:

In [ ]:
query_filter = {
  "@context": "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/data/v1.0.4",
  "filter": {
    "op": "eq",
    "path": "rdf:type",
    "value": "nsg:Neuron"
  }
}

Execute the code below to see how many instances of type nsg:Neuron are available to you:

In [ ]:
response = requests.post("https://bbp-nexus.epfl.ch/staging/v0/queries/sandbox", 
                         json=query_filter, 
                         allow_redirects=False,
                         headers={"Authorization": "Bearer {}".format(token)})
get_response = requests.get(response.headers["Location"])
print("There are a total of {} instances of type nsg:Neuron in the sandbox organization of Nexus".format(yaml.load(get_response.text)["total"]))

**THE END**